<a href="https://colab.research.google.com/github/tbuzze/COVID-19/blob/master/MedeZUP_JUL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise MedeZUP

In [ ]:
#-- carregando autorização
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
#-- instalando plotly
!pip install plotly
!pip install chart-studio

#-- importando libs
import pandas as pd
import plotly
import chart_studio.plotly as py

In [ ]:
#-- conectando ao bigquery
%%bigquery --project zup-intelligence df
SELECT *
FROM `zup-intelligence.raw_people_pesquisas.dl_pesquisa_medezup_jul2020` 

In [ ]:
#-- printando o head
df.head()

,dt_pesquisa,nm_pod,ds_idade,ds_alocacao,ds_tempo_empresa,rt_evolucao_pos_ameaca_saida,rt_intra_empreendedor,rt_perda_foco,rt_aprendizado_por_referencias_mercado,rt_liberdade_opinioes,rt_canais_corretos,rt_modos_ritos_aplicados,rt_deconhecer_diretrizes_seguranca,rt_time_visionario,rt_motivar_ambiente_confianca,rt_aprendizado_rapido,rt_zuppers_individualistas,rt_experimentar_solucoes_diferentes,rt_informacao_erros_acessivel,rt_receio_opinar,rt_divisao_preocupacoes,rt_compartilho_erros,rt_nao_estudar,rt_desafios_complexos_colaborativos,rt_falta_visao_projeto,rt_testes_rapidos,rt_ascensao_nivel_tecnico_recente,rt_contratacoes_aumentam_barra_time,rt_objetivo_claro_time_indicadores,rt_time_compartilha_conhecimento,rt_empatia_orientacao_zuppers,rt_dificuldade_trabalho_time,rt_consenso_time,rt_foco_causa,rt_ensinamentos_desafios_atritos_positivos,rt_preferencia_lentidao_e_certeza,rt_discussoes_tecnicas_profundas,rt_objetivo_claro_projeto,rt_seguranca_informacao_abordada_constantemente,rt_diversidade_time,rt_atencao_LGPD,rt_horarios_respeitados,rt_falta_de_respeito_zuppers,rt_recebo_feedbacks_necessarios,rt_falta_colaboratividade_zuppers,rt_confortabilidade_pedir_auxilio,rt_estimulo_respeitar_opinioes_diversas,rt_obcecados_entender_relevancia_para_cliente,rt_tecnologias_processos_eficazes,rt_crescimento_exponencial_encorajado,rt_dificuldade_acompanhar_mercado,rt_trabalho_metrificado,rt_zuppers_confiantes_motivados_empaticos,rt_voz_ativa_sem_excecao,rt_protecao_dados_sensiveis_clientes,rt_lider_proativo,rt_lider_compreensivel,rt_home_office_permite_cultura_zup,rt_vive_proposito_e_treze_pilares,rt_leu_book_cultura_zup,rt_mudanca_comportamento_nova_cultura_zup,ds_complemento_respostas,ds_ideia_acao,ds_palavras_chave_ZUP,ds_email_referencia_tecnica,ds_email_referencia_saida
0,2020-07-29 14:32:45,Itaú - Cartões,De 25 a 29 anos,ZUP,Entre 6 meses e 1 ano,1.0,4.0,2.0,3.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,2.0,4.0,3.0,1.0,2.0,4.0,3.0,4.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,4.0,4.0,4.0,3.0,4.0,3.0,5.0,5.0,5.0,5.0,1.0,5.0,1.0,5.0,4.0,3.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,5.0,5.0,5.0,4.0,3.0,3.0,Nada a acrescentar,Nada a acrescentar,"Crescimento Exponencial, Inovação, Segurança",paulo.carrara@zup.com.br,
1,2020-07-29 17:21:28,Itaú - Cartões,40 anos ou mais,ZUP,Menos de 1 mês,1.0,3.0,2.0,2.0,4.0,4.0,3.0,1.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,3.0,2.0,4.0,1.0,4.0,4.0,3.0,3.0,4.0,4.0,1.0,4.0,3.0,3.0,4.0,3.0,4.0,5.0,5.0,5.0,3.0,1.0,4.0,3.0,5.0,4.0,4.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,,,"Eficácia, Inclusão e Inovação",,
2,2020-07-29 09:46:20,Key Accounts - Pod UAI,40 anos ou mais,Cliente,Entre 1 ano e 2 anos,2.0,4.0,1.0,4.0,3.0,4.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,4.0,2.0,3.0,5.0,2.0,5.0,1.0,4.0,4.0,3.0,4.0,4.0,5.0,1.0,5.0,5.0,3.0,1.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,4.0,1.0,5.0,5.0,4.0,3.0,4.0,1.0,3.0,4.0,5.0,5.0,4.0,5.0,5.0,3.0,5.0,5.0,Quem fica alocado no cliente não vive a cultur...,Investir em treinamentos e inglês. Buscar proj...,Desafio,Tiago.Lopes@zup.com.br,
3,2020-07-29 12:29:33,Key Accounts - Pod UAI,De 30 a 34 anos,Cliente,Entre 6 meses e 1 ano,1.0,3.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,1.0,5.0,1.0,4.0,4.0,3.0,2.0,3.0,4.0,3.0,1.0,1.0,4.0,4.0,5.0,4.0,5.0,1.0,5.0,4.0,4.0,2.0,4.0,4.0,3.0,5.0,4.0,5.0,1.0,5.0,1.0,5.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,Por estar alocado no cliente a cultura Zup che...,"Uma sugestão, para pós pandemia, é aumentar o ...","crescimento, aprendizado, oportunidade, transf...",jefferson.pereira@zup.com.br,jefferson.pereira@zup.com.br
4,2020-07-29 10:13:31,Key Accounts - Pod UAI,De 25 a 29 anos,ZUP,Entre 1 ano e 2 anos,3.0,4.0,3.0,4.0,3.0,3.0,2.0,3.0,3.0,5.0,4.0,2.0,4.0,5.0,3.0,3.0,3.0,2.0,2.0,2.0,4.0,5.0,3.0,3.0,4.0,4.0,2.0,2.0,3.0,4.0,2.0,4.0,3.0,2.0,4.0,4.0,5.0,1.0,5.0,1.0,4.0,4.0,3.0,2.0,5.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,3.0,,,"liberdade, aprendizado constante",,


In [ ]:
#-- estatísticas básicas do df
df.describe()

,rt_evolucao_pos_ameaca_saida,rt_intra_empreendedor,rt_perda_foco,rt_aprendizado_por_referencias_mercado,rt_liberdade_opinioes,rt_canais_corretos,rt_modos_ritos_aplicados,rt_deconhecer_diretrizes_seguranca,rt_time_visionario,rt_motivar_ambiente_confianca,rt_aprendizado_rapido,rt_zuppers_individualistas,rt_experimentar_solucoes_diferentes,rt_informacao_erros_acessivel,rt_receio_opinar,rt_divisao_preocupacoes,rt_compartilho_erros,rt_nao_estudar,rt_desafios_complexos_colaborativos,rt_falta_visao_projeto,rt_testes_rapidos,rt_ascensao_nivel_tecnico_recente,rt_contratacoes_aumentam_barra_time,rt_objetivo_claro_time_indicadores,rt_time_compartilha_conhecimento,rt_empatia_orientacao_zuppers,rt_dificuldade_trabalho_time,rt_consenso_time,rt_foco_causa,rt_ensinamentos_desafios_atritos_positivos,rt_preferencia_lentidao_e_certeza,rt_discussoes_tecnicas_profundas,rt_objetivo_claro_projeto,rt_seguranca_informacao_abordada_constantemente,rt_diversidade_time,rt_atencao_LGPD,rt_horarios_respeitados,rt_falta_de_respeito_zuppers,rt_recebo_feedbacks_necessarios,rt_falta_colaboratividade_zuppers,rt_confortabilidade_pedir_auxilio,rt_estimulo_respeitar_opinioes_diversas,rt_obcecados_entender_relevancia_para_cliente,rt_tecnologias_processos_eficazes,rt_crescimento_exponencial_encorajado,rt_dificuldade_acompanhar_mercado,rt_trabalho_metrificado,rt_zuppers_confiantes_motivados_empaticos,rt_voz_ativa_sem_excecao,rt_protecao_dados_sensiveis_clientes,rt_lider_proativo,rt_lider_compreensivel,rt_home_office_permite_cultura_zup,rt_vive_proposito_e_treze_pilares,rt_leu_book_cultura_zup,rt_mudanca_comportamento_nova_cultura_zup
count,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.00000,324.000000,324.000000,324.000000,324.000000,324.000000,324.00000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,2.222222,3.416667,3.132716,3.811728,4.064815,3.632716,3.598765,2.166667,3.697531,4.330247,4.256173,2.635802,3.910494,3.632716,2.753086,2.564815,3.728395,2.941358,3.851852,1.805556,3.209877,4.003086,3.929012,3.469136,4.160494,4.348765,1.552469,3.851852,3.447531,4.095679,2.929012,3.956790,3.645062,4.265432,4.354938,4.123457,3.944444,1.320988,4.12963,1.697531,4.503086,4.265432,3.950617,3.854938,4.37963,1.984568,3.342593,3.910494,4.583333,4.567901,4.070988,4.237654,4.237654,3.987654,3.493827,3.250000
std,1.104452,1.065574,1.189861,1.003886,0.982256,1.015809,1.101420,1.130769,1.020469,0.882691,0.750104,1.054292,0.948201,1.166208,1.290891,1.072653,1.124039,1.278406,1.030328,1.071370,1.241352,1.105848,1.021998,1.264778,0.900042,0.816982,0.957467,1.024301,1.061729,0.843907,1.034045,1.114769,1.140558,0.962409,0.894050,0.965443,1.227901,0.739461,1.04475,0.954634,0.849325,0.926348,1.063335,1.001828,0.82961,1.100131,1.152427,1.020539,0.837862,0.711485,1.120271,1.014905,1.027035,0.960441,1.494041,1.133162
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,3.000000,2.000000,3.000000,4.000000,3.000000,3.000000,1.000000,3.000000,4.000000,4.000000,2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,3.000000,3.000000,3.000000,4.000000,4.000000,1.

In [ ]:
#-- criando a função do gráfico de Sankey
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [ ]:
#-- criando o agrupamento
df_group = df.groupby(['nm_pod', 'ds_idade', 	'ds_alocacao', 	'ds_tempo_empresa'])['rt_evolucao_pos_ameaca_saida'].count().reset_index()

In [ ]:
#-- printando o gráfico
fig = genSankey(df_group,cat_cols=['nm_pod', 'ds_idade', 	'ds_alocacao', 	'ds_tempo_empresa'],value_cols='rt_evolucao_pos_ameaca_saida',title='Sankey Diagram')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'